# MTA Analysis: Data Acquisition & Cleaning
---

### Setup
#### Import required libraries.

In [1]:
import pandas as pd
import numpy as np
import os

#### Define useful functions (maybe document and put this in .py file later?)

In [2]:
def convert_timestamp_to_mta_format(timestamp):
    """
    """
    year = str(timestamp.year)[2:4]
    month = str(timestamp.month).zfill(2)
    day = str(timestamp.day).zfill(2)
    
    date_mta_format = year + month + day
    
    return date_mta_format

def get_hourly_counts(row, max_counter):
    """
    """
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # Maybe counter is reversed?
        counter = -counter
    if counter > max_counter:
        # Maybe counter was reset to 0? 
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

### Data Acquisition
Load [MTA Turnstile Data](http://web.mta.info/developers/turnstile.html) and combine into a single dataframe.
_Note: The files already exist in the repository. This code block can be skipped over._

In [3]:
start_date = '2015-05-02'
months_of_interest = [5, 6]
start_ts = pd.Timestamp(start_date)

In [4]:
current_timestamp = start_ts
is_date_valid = True
raw_master_df = pd.DataFrame()
while is_date_valid:
    if current_timestamp.month not in months_of_interest:
        current_timestamp += pd.DateOffset(days=7)
        continue
    
    print(f'Downloading data for {current_timestamp}...')
    date_formatted = convert_timestamp_to_mta_format(current_timestamp)
    
    # load data and write to csv
    url = f'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{date_formatted}.txt'
    df_turnstile_data = pd.read_csv(url)
    raw_master_df = pd.concat([raw_master_df, df_turnstile_data])
    
    # add 7 days to get next file. if resulting date is later than today, then stop loop
    current_timestamp += pd.DateOffset(days=7)
    if current_timestamp > pd.to_datetime("now"):
        is_date_valid = False

Export combined dataframe to pickle.

In [5]:
raw_master_df.to_pickle('raw_mta_turnstile_data_mayjune_20152020.pkl')

### Data Cleaning
Load pickled data.

In [6]:
df_mta_raw = pd.read_pickle('raw_mta_turnstile_data_mayjune_20152020.pkl')
df_mta_raw.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,REGULAR,5106770,1729635
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,REGULAR,5106810,1729649
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,REGULAR,5106835,1729680
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,REGULAR,5106961,1729784
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,REGULAR,5107250,1729858


Remove any whitespace from column names and initialize a new dataframe to store cleaned data.

In [7]:
df_mta_clean = df_mta_raw
df_mta_clean.columns = df_mta_clean.columns.str.replace(' ', '')

Determine if there are any NaN values.

In [11]:
df_mta_raw[df_mta_clean.isna()].count()

C/A         0
UNIT        0
SCP         0
STATION     0
LINENAME    0
DIVISION    0
DATE        0
TIME        0
DESC        0
ENTRIES     0
EXITS       0
dtype: int64

No NaN values to filter.

Add `DATETIME` column to dataset.

In [14]:
df_mta_clean['DATETIME'] = pd.to_datetime(df_mta_clean['DATE'] + ' ' + df_mta_clean['TIME'])

Check if there are duplicates by determining if there are multiple readings for a given turnstile at a given time.

In [18]:
df_checking_duplicates = (df_mta_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"]).ENTRIES.count().reset_index()).sort_values('ENTRIES')
df_checking_duplicates

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2016-04-30 00:00:00,1
6896126,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 04:00:00,1
6896127,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 08:00:00,1
6896128,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 12:00:00,1
6896129,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-26 16:00:00,1
...,...,...,...,...,...,...
6410336,R101,R001,02-00-00,SOUTH FERRY,2018-05-23 09:00:00,2
2073816,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-14 05:00:00,2
4774408,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-16 16:00:00,2
4774415,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-17 08:00:00,2


In [19]:
df_checking_duplicates = df_checking_duplicates[df_checking_duplicates['ENTRIES'] == 2]
df_checking_duplicates

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
9359628,R518,R261,00-03-02,40 ST LOWERY ST,2017-06-05 08:00:00,2
5189785,N525,R142,01-00-03,DELANCEY/ESSEX,2019-05-11 05:00:00,2
811622,B020,R263,00-06-01,AVENUE H,2016-05-30 12:00:00,2
4290068,N329,R201,00-03-04,WOODHAVEN BLVD,2015-05-20 05:00:00,2
8513239,R290,R161,00-00-00,KINGSBRIDGE RD,2019-06-07 05:00:00,2
...,...,...,...,...,...,...
6410336,R101,R001,02-00-00,SOUTH FERRY,2018-05-23 09:00:00,2
2073816,JFK03,R536,00-00-05,JFK JAMAICA CT1,2015-05-14 05:00:00,2
4774408,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-16 16:00:00,2
4774415,N418,R269,01-05-00,BEDFORD-NOSTRAN,2016-05-17 08:00:00,2


In [ ]:
# TODO use a merge to get relevant rows from original df

Check to see what is causing the duplicate.

In [23]:
mask = (
    (df_mta_clean['C/A'] == 'R518') &
    (df_mta_clean['UNIT'] == 'R261') &
    (df_mta_clean['SCP'] == '00-03-02') &
    (df_mta_clean['STATION'] == '40 ST LOWERY ST') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2017-06-05 08:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
178845,R518,R261,00-03-02,40 ST LOWERY ST,7,IRT,06/05/2017,08:00:00,REGULAR,18778966,7453831,2017-06-05 08:00:00
178846,R518,R261,00-03-02,40 ST LOWERY ST,7,IRT,06/05/2017,08:00:00,RECOVR AUD,18778965,7453831,2017-06-05 08:00:00


Spot check randomly to see if REGULAR and RECOVR AUD readings are different.

In [28]:
mask = (
    (df_mta_clean['C/A'] == 'R290') &
    (df_mta_clean['UNIT'] == 'R161') &
    (df_mta_clean['SCP'] == '00-00-00') &
    (df_mta_clean['STATION'] == 'KINGSBRIDGE RD') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2019-06-07 05:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
169384,R290,R161,00-00-00,KINGSBRIDGE RD,4,IRT,06/07/2019,05:00:00,REGULAR,183445,164296,2019-06-07 05:00:00
169385,R290,R161,00-00-00,KINGSBRIDGE RD,4,IRT,06/07/2019,05:00:00,RECOVR AUD,183444,164296,2019-06-07 05:00:00


In [29]:
mask = (
    (df_mta_clean['C/A'] == 'N045') &
    (df_mta_clean['UNIT'] == 'R187') &
    (df_mta_clean['SCP'] == '01-00-02') &
    (df_mta_clean['STATION'] == '81 ST-MUSEUM') &
    (df_mta_clean['DATETIME'] == pd.to_datetime('2019-06-17 09:00:00'))
)
df_mta_clean[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
49121,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/17/2019,09:00:00,REGULAR,4343886,994027,2019-06-17 09:00:00
49122,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/17/2019,09:00:00,RECOVR AUD,4343885,994027,2019-06-17 09:00:00


In [24]:
df_mta_clean.DESC.value_counts()

REGULAR       10303603
RECOVR AUD       40700
Name: DESC, dtype: int64

Based on a spot check, it appears that where there are duplicates, the REGULAR and RECOVR AUD readings are identical.
We will assume that we can drop any duplicate entries (regardless of the type).

In [31]:
df_mta_clean.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"], inplace=True)

Confirm no duplicates.

In [32]:
(df_mta_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATETIME"]).ENTRIES.count().reset_index()).sort_values('ENTRIES')

,C/A,UNIT,SCP,STATION,DATETIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2016-04-30 00:00:00,1
6896138,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 04:00:00,1
6896139,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 08:00:00,1
6896140,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 12:00:00,1
6896141,R145,R032,00-00-00,TIMES SQ-42 ST,2018-05-28 16:00:00,1
...,...,...,...,...,...,...
3448070,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-25 17:00:00,1
3448071,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-25 21:00:00,1
3448072,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-26 01:00:00,1
3448065,N135,R385,01-03-01,ROCKAWAY BLVD,2015-06-24 21:00:00,1


Drop columns `DESC`. Supress errors.

In [34]:
df_mta_clean = df_mta_clean.drop("DESC", axis=1, errors="ignore")
df_mta_clean.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME
0,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,00:00:00,5106770,1729635,2015-04-25 00:00:00
1,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,04:00:00,5106810,1729649,2015-04-25 04:00:00
2,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,08:00:00,5106835,1729680,2015-04-25 08:00:00
3,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,12:00:00,5106961,1729784,2015-04-25 12:00:00
4,A002,R051,02-00-00,LEXINGTON AVE,NQR456,BMT,04/25/2015,16:00:00,5107250,1729858,2015-04-25 16:00:00


Turnstile counts (entries and exits) are cumulative. In order to actually calculate the number of entries/exits per timestamp, some calculations need to be performed.

In [ ]:
df_mta_clean[["PREV_DATETIME", "PREV_ENTRIES", "PREV_EXITS"]] = (
    df_mta_clean
    .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATETIME", "ENTRIES", "EXITS"]
    .apply(lambda grp: grp.shift(1)))

df_mta_clean.head()

<ipython-input-41-fefb254bebf5>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_mta_clean


In [ ]:
df_mta_clean.to_pickle('df_mta_clean_after_shift.pkl')

TODO: Figure out how to deal with resets.

TODO: Figure out how to deal with reverse counting.